In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score # AUC score
from sklearn.metrics import average_precision_score # AUPR score
from sklearn.metrics import precision_recall_fscore_support # precision, recall
from imblearn.metrics import sensitivity_specificity_support # sensitivity, specificity
from sklearn.metrics import roc_curve # to draw auc curve
from sklearn.metrics import precision_recall_curve # to draw aupr curve
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import NearMiss
from itertools import cycle
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#2017.3.27. PREDICT independent dataset
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/PREDICT_data/2_2_indep_descriptor.tsv")
indep_x = indep.values[:, 3:].astype(float)
indep_y = indep.values[:,2].astype(int)

In [3]:
#2017.3.30. PREDICT TN_local + noise dataset
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/PREDICT_data/3_3_2_tn_local_noise_descriptor_predict.tsv")
x_tnNoise = df[df.columns.values[3:].tolist()].values
y_tnNoise = df["association"].values

In [137]:
indep_new_drug.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/PREDICT_data/3_3_indep_new_drug.tsv", sep='\t', index=False)

In [140]:
indep_new_drug_asso[['drug_id', 'disease_id', 'association']].to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/PREDICT_data/3_5_indep_new_drug_asso.tsv", sep='\t', index=False)

In [7]:
#2017.4.3. PREDICT no TN_local
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/PREDICT_data/3_3_tn_local_noise_descriptor_predict.tsv")
x_noTN = df[df.columns.values[3:].tolist()].values
y_noTN = df["association"].values

In [6]:
# 2017.4.3.TN_local + noise 로 train한 모델로 indepndent 예측하는 양상 보기
clf_tnNoise = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight='balanced')
clf_tnNoise.fit(x_tnNoise, y_tnNoise)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
# 2017.4.3.noTN으로 train한 모델로 indepndent 예측하는 양상 보기
clf_noTN = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight='balanced')
clf_noTN.fit(x_noTN, y_noTN)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
indep.head()

,drug_id,disease_id,association,C0000727,C0000731,C0000737,C0000768,C0000772,C0000786,C0000814,...,387585,388165,55219,55973,65264,80350,84624,84722,91851,406216
0,DB01200,102300,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DB01200,103780,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DB01200,104300,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DB01200,115195,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DB01200,137580,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
print clf_tnNoise.predict_proba(indep_x[0].reshape(1,-1))
print clf_noTN.predict_proba(indep_x[0].reshape(1,-1))

[[ 0.974  0.026]]
[[ 0.954  0.046]]


In [33]:
print len(indep)

2539


In [35]:
df_result =pd.DataFrame()
cnt = 0
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in indep.iterrows():
    drug_id = row['drug_id']
    disease_id = row['disease_id']
    y = row['association']
    x = row.values[3:].reshape(1,-1)
   
    tnNoise_proba = clf_tnNoise.predict_proba(x)
    tnNoise_label = clf_tnNoise.predict(x)
    noTN_proba =
    clf_noTN.predict_proba(x)
    noTN_label = clf_noTN.predict(x)
    temp_df = pd.DataFrame(data = [[drug_id, disease_id, y, tnNoise_label, tnNoise_proba[:,1][0], noTN_label, noTN_proba[:,1][0]]],
                           columns=['drug_id', 'disease_id', 'y', 'tnNoise_label', 'tnNoise_proba', 'noTN_label','noTN_proba'])
    df_result = df_result.append(temp_df)
    cnt = cnt + 1
    if cnt == 0:
        print "cnt : {}, ".format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')

2017-04-03 19:01:07
2017-04-03 20:16:17


In [38]:
temp = list()
for i in df_result['tnNoise_label']:
    temp.append(i[0])

In [40]:
temp2 = list()
for i in df_result['noTN_label']:
    temp2.append(i[0])

In [42]:
df_temp = df_result

In [43]:
df_temp['tnNoise_label'] = temp
df_temp['noTN_label'] = temp2

In [97]:
df_temp.index = range(0, len(df_temp))
df_temp.head(2)

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
0,DB01200,102300,1,0,0.026,0,0.046
1,DB01200,103780,0,0,0.040,0,0.062


In [153]:
df_temp.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/PREDICT_data/noTn_vs_noise/independent_1_by_1.tsv", sep='\t', index=False)

In [57]:
# fp
fp_tnNoise = df_temp[(df_temp.y != df_temp.tnNoise_label) & (df_temp.tnNoise_label == 1)]
fp_noTN = df_temp[(df_temp.y != df_temp.noTN_label) & (df_temp.noTN_label == 1)]

In [58]:
# fn
fn_tnNoise = df_temp[(df_temp.y != df_temp.tnNoise_label) & (df_temp.tnNoise_label == 0)]
fn_noTN = df_temp[(df_temp.y != df_temp.noTN_label) & (df_temp.noTN_label == 0)]

In [59]:
# tn
tn_tnNoise = df_temp[(df_temp.y == df_temp.tnNoise_label) & (df_temp.tnNoise_label == 0)]
tn_noTN = df_temp[(df_temp.y == df_temp.noTN_label) & (df_temp.noTN_label == 0)]

In [60]:
# tp
tp_tnNoise = df_temp[(df_temp.y == df_temp.tnNoise_label) & (df_temp.tnNoise_label == 1)]
tp_noTN = df_temp[(df_temp.y == df_temp.noTN_label) & (df_temp.noTN_label == 1)]

In [106]:
path = "/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/PREDICT_data/"
fp_tnNoise.to_csv(path + "noTn_vs_noise/fp_tnNoise.tsv", sep='\t')
fp_noTN.to_csv(path + "noTn_vs_noise/fp_noTN.tsv", sep='\t')
fn_tnNoise.to_csv(path + "noTn_vs_noise/fn_tnNoise.tsv", sep='\t')
fn_noTN.to_csv(path + "noTn_vs_noise/fn_noTN.tsv", sep='\t')
tn_tnNoise.to_csv(path + "noTn_vs_noise/tn_tnNoise.tsv", sep='\t')
tn_noTN.to_csv(path + "noTn_vs_noise/tn_noTN.tsv", sep='\t')
tp_tnNoise.to_csv(path + "noTn_vs_noise/tp_tnNoise.tsv", sep='\t')
tp_noTN.to_csv(path + "noTn_vs_noise/tp_noTN.tsv", sep='\t')

In [70]:
print 'fp : {}, {}'.format(len(fp_noTN), len(fp_tnNoise))
print 'fn : {}, {}'.format(len(fn_noTN), len(fn_tnNoise))
print 'tn : {}, {}'.format(len(tn_noTN), len(tn_tnNoise))
print 'tp : {}, {}'.format(len(tp_noTN), len(tp_tnNoise))

fp : 14, 15
fn : 83, 82
tn : 2432, 2431
tp : 10, 11


In [84]:
# no TN만 Fn인거
fn_noTN[~fn_noTN.index.isin(fn_tnNoise.index)]

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
1835,DB00635,603165,1,1,0.658,0,0.442


In [86]:
# no TN만 Fp인거
fp_noTN[~fp_noTN.index.isin(fp_tnNoise.index)]

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
937,DB00810,104300,0,0,0.49,1,0.506


In [85]:
# noise만 FP인거
fp_tnNoise[~fp_tnNoise.index.isin(fp_noTN.index)]

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
1828,DB00635,256370,0,1,0.502,0,0.396
1985,DB00253,137760,0,1,0.606,0,0.458


In [101]:
# 공통 fp
union_fp = fp_tnNoise[fp_tnNoise.index.isin(fp_noTN.index)]
union_fp

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
14,DB01200,168600,0,1,0.706000,1,0.726000
594,DB01586,140600,0,1,0.531324,1,0.533900
599,DB01586,165720,0,1,0.531324,1,0.533900
621,DB01586,607850,0,1,0.531324,1,0.533900
668,DB00959,165720,0,1,0.713208,1,0.686000
690,DB00959,607850,0,1,0.713208,1,0.686000
1038,DB01173,607501,0,1,0.651191,1,0.665461
1351,DB00313,607501,0,1,0.634381,1,0.632652
1357,DB00909,137580,0,1,0.664000,1,0.630000
1701,DB00434,607501,0,1,0.625045,1,0.752219


In [107]:
union_fp.to_csv(path + "noTn_vs_noise/union_fp.tsv", sep='\t')

In [109]:
indep[indep.index.isin(union_fp.index)].to_csv(path + "noTn_vs_noise/union_fp_descriptor.tsv", sep='\t')

In [104]:
tp_noTN

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
772,DB00461,165720,1,1,0.644000,1,0.654000
794,DB00461,607850,1,1,0.644000,1,0.654000
914,DB00376,168600,1,1,0.650000,1,0.644000
949,DB00810,168600,1,1,0.676000,1,0.702000
984,DB00915,168600,1,1,0.694000,1,0.730000
2397,DB00320,607501,1,1,0.641844,1,0.705900
2432,DB00669,607501,1,1,0.618761,1,0.667809
2467,DB00952,607501,1,1,0.693342,1,0.649818
2502,DB00953,607501,1,1,0.632000,1,0.652000
2537,DB00998,607501,1,1,0.678641,1,0.647456


In [96]:
# 공통 fn
fn_noTN[fn_noTN.index.isin(fn_tnNoise.index)]

,drug_id,disease_id,y,tnNoise_label,tnNoise_proba,noTN_label,noTN_proba
0,DB01200,102300,1,0,0.026000,0,0.046000
35,DB00575,103780,1,0,0.182000,0,0.150000
70,DB01037,104300,1,0,0.098000,0,0.094000
82,DB01037,168600,1,0,0.202000,0,0.198000
106,DB00280,115195,1,0,0.002000,0,0.008000
141,DB00343,115195,1,0,0.042000,0,0.046000
176,DB01100,137580,1,0,0.058000,0,0.062000
212,DB00819,137750,1,0,0.082000,0,0.032000
213,DB00819,137760,1,0,0.058000,0,0.050000
246,DB01085,137750,1,0,0.056000,0,0.078000


In [ ]:
def Random_Forest_independent_graph(X, y, indep_X, indep_y, user_estimator):

    folds_results = dict()
    draw_results = {'fpr':[], 'tpr':[], 'precision_vec':[], 'recall_vec':[]}
    fp_results = dict()
    
    classifier = RandomForestClassifier(n_estimators=user_estimator, n_jobs=-1, class_weight='balanced')
    classifier.fit(X, y)
    indep_y_predicted_proba = classifier.predict_proba(indep_X)
    indep_y_predicted_label = classifier.predict(indep_X)
    fp_results['predicted_proba'] = indep_y_predicted_proba
    fp_results['Predicted_label'] = indep_y_predicted_label
    fp_results['y_true'] = indep_y

    # Accuracy
    indep_acc = classifier.score(indep_X, indep_y)
    folds_results['acc']=indep_acc
    
    # AUC
    indep_auc = roc_auc_score(indep_y, indep_y_predicted_proba[:,1])
    folds_results['auc']=indep_auc
    
    # Sensitivity, Specificity
    indep_sn, indep_sp, support = sensitivity_specificity_support(indep_y, indep_y_predicted_label)
    folds_results['sn']=indep_sn
    folds_results['sp']=indep_sp
    
    # AUPR
    indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:,1])
    folds_results['aupr']=indep_aupr
    
    # Precision, Recall
    indep_precision, indep_recall, _, _ = precision_recall_fscore_support(indep_y, indep_y_predicted_label, average = 'binary')
    folds_results['precision']=indep_precision
    folds_results['recall']=indep_recall
    
    # Confusion Matirx
    indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
    folds_results['confusion_matrix']=indep_confusion_matirx
    
    # draw graph
    indep_fpr, indep_tpr, thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label=1)
    draw_results['fpr'].append(indep_fpr)
    draw_results['tpr'].append(indep_tpr)
        
    indep_precision_vec, indep_recall_vec, _ = precision_recall_curve(indep_y, indep_y_predicted_proba[:, 1])
    draw_results['precision_vec'].append(indep_precision_vec)
    draw_results['recall_vec'].append(indep_recall_vec)

    return folds_results, draw_results, fp_results

In [ ]:
# Random_Forest_independent_graph + 파일 입출력 + 1:1
# PREDICT dataset local no tn indep , indep1 10-fold, 1:1  2017.4.3. 작성
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
path = '/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/PREDICT_data/'
with open(path + "2_9_RF_no_tn_indep.txt", 'w') as fd :
    user_estimator = 500
    
    fd.write("<independent>\r\n")
    folds_results, draw_results, fp_results = Random_Forest_independent_graph(x_whole_data, y_whole_data, indep_x, indep_y, user_estimator)
    draw_plot([draw_results], folds_results, user_estimator)
    write_output(fd, folds_results, user_estimator)
    #--------------------------------------------------------------
    fd.write("<1:1 independent>\r\n")
    independent_count = 10
    indep_results = dict()
    indep_draw = list()
    for i in range(independent_count):
        x_resampled, y_resampled = rus.fit_sample(x_whole_data, y_whole_data)
        folds_results2, draw_results2, indep_results2= Random_Forest_independent_graph(x_resampled, y_resampled, indep_x, indep_y, user_estimator)
        indep_results[i] = folds_results2
        indep_draw.append(draw_results2)    
    seperated = seperate_results(indep_results)
    final_result = average_result(seperated)
    draw_plot(indep_draw, final_result, user_estimator)
    write_output(fd, final_result, user_estimator)    
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')